In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import calendar
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
# Load data and convert date columns to datetime
df = pd.read_excel("DS Internship - EDA - Data.xlsx")
date_columns = ['Store Open', 'Store Close', 'Store Modification Date']
for col in date_columns:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

In [3]:
# ----- INITIAL EXPLORATION -----

# 1a. Find total sales by year
yearly_sales = df.groupby('Year')['Sales'].sum().reset_index()
print(yearly_sales)

   Year         Sales
0  2015  1.627585e+09
1  2016  1.843939e+09
2  2017  1.651322e+09
3  2018  1.492340e+09
4  2019  1.471963e+09
5  2020  6.510899e+07


In [4]:
# 1b. Number of stores opened in the year 1991
stores_1991 = df[df['Store Open'].dt.year == 1991]['Store'].nunique()
print(stores_1991)

4


In [5]:
# 1c. How many stores were remodelled during this period?
remodelled_stores = df[~df['Store Modification Date'].isna()]['Store'].nunique()
print(remodelled_stores)

346


In [6]:
# 1d. Find the direct relationship between Sales and total sq. ft.
store_df = df.groupby('Store').agg({'Sales': 'mean', 'Total Sq Ft': 'first'}).reset_index()
sales_sqft_corr = store_df['Sales'].corr(store_df['Total Sq Ft'])
print(sales_sqft_corr)

0.4750814572072202


In [7]:
# 1e. Which Super division is most profitable?
division_sales = df.groupby('Super Division')['Sales'].sum().sort_values(ascending=False)
print(division_sales)

Super Division
GIRLS          3.572426e+09
BOYS           2.018241e+09
ACCESSORIES    1.821397e+09
KIDS           7.401937e+08
Name: Sales, dtype: float64


In [8]:
# 1f. How many stores are active as of today?
active_stores = df[df['Store Close'].isna() | (df['Store Close'] > datetime.now())]['Store'].nunique()
print(active_stores)

678


In [9]:
# 1g. Which super division has more sq. ft on average?
# Get each store's first Super Division and sq. ft.
store_sqft = df.groupby('Store').agg({'Super Division': 'first', 'Total Sq Ft': 'first'}).reset_index()
avg_sqft_by_division = store_sqft.groupby('Super Division')['Total Sq Ft'].mean().reset_index()
print(avg_sqft_by_division.sort_values('Total Sq Ft', ascending=False))

  Super Division  Total Sq Ft
3           KIDS  7785.300000
0    ACCESSORIES  7341.650000
1           BOYS  7082.277778
2          GIRLS  6243.321410


In [10]:
# ----- ADVANCED INSIGHTS -----

# 2a. Top 3 candidate states for new stores
state_avg_sales = df.groupby(['State', 'Store'])['Sales'].mean().groupby('State').mean().reset_index()
state_store_count = df.groupby('State')['Store'].nunique().reset_index().rename(columns={'Store': 'Store Count'})
state_analysis = state_avg_sales.merge(state_store_count, on='State')

# Simplified scoring: Normalize factors and create composite score (70% sales, 30% low store count)
state_analysis['Normalized Sales'] = (state_analysis['Sales'] - state_analysis['Sales'].min()) / (state_analysis['Sales'].max() - state_analysis['Sales'].min())
state_analysis['Normalized Store Count'] = 1 - ((state_analysis['Store Count'] - state_analysis['Store Count'].min()) / (state_analysis['Store Count'].max() - state_analysis['Store Count'].min()))
state_analysis['Composite Score'] = state_analysis['Normalized Sales'] * 0.7 + state_analysis['Normalized Store Count'] * 0.3

top_states = state_analysis.sort_values('Composite Score', ascending=False).head(3)

print(top_states[['State', 'Sales', 'Store Count', 'Composite Score']])

   State         Sales  Store Count  Composite Score
43    E4  58183.380314            2         0.996341
15    B5  49794.577592            3         0.796508
20    C1  49510.719503            7         0.775236


In [11]:
# 2b. Best time of year to open a store
# Find Average sales by Month
monthly_sales = df.groupby('Month')['Sales'].mean().reset_index()
month_names = {i: calendar.month_name[i] for i in range(1, 13)}
monthly_sales['Month Name'] = monthly_sales['Month'].map(month_names)

print(monthly_sales.sort_values('Sales', ascending=False))

    Month         Sales Month Name
11     12  63392.905721   December
7       8  57362.934328     August
2       3  48179.098097      March
10     11  47063.812595   November
6       7  45525.699765       July
3       4  43248.803190      April
5       6  42861.734267       June
4       5  41712.454066        May
1       2  40888.945072   February
8       9  39106.330348  September
9      10  38728.562158    October
0       1  31716.908792    January


In [12]:
# 2c. Outlet-type effects on store closures
df['Is Closed'] = ~df['Store Close'].isna()
outlet_closure = df.groupby(['Outlet Type', 'Store'])['Is Closed'].first().groupby('Outlet Type').mean().reset_index()
outlet_closure.columns = ['Outlet Type', 'Closure Rate']
outlet_closure['Closure Rate'] *= 100  # Convert to percentage

# Store Closure Rate by Outlet Type
print(outlet_closure.sort_values('Closure Rate', ascending=False))

  Outlet Type  Closure Rate
1      Outlet     33.636364
2       Strip     23.442136
0        Mall      9.399478


In [13]:
# Calculate average lifespan for closed stores
closed_stores = df[~df['Store Close'].isna()].copy()
if not closed_stores.empty:
    closed_stores['Store Lifespan'] = (closed_stores['Store Close'] - closed_stores['Store Open']).dt.days / 365.25
    outlet_lifespan = closed_stores.groupby(['Outlet Type', 'Store'])['Store Lifespan'].first().groupby('Outlet Type').mean().reset_index()
    
    print(outlet_lifespan.sort_values('Store Lifespan', ascending=False))

  Outlet Type  Store Lifespan
1      Outlet       19.769725
0        Mall       15.286029
2       Strip       10.270579


In [14]:
print("\n----- SUMMARY OF KEY INSIGHTS -----")
print(f"1. Total number of unique stores in the dataset: {df['Store'].nunique()}")
print(f"2. Most profitable Super Division: {division_sales.index[0]}")
print(f"3. Top 3 states recommended for new stores: {', '.join(top_states['State'].tolist())}")
print(f"4. Best month to open a store: {monthly_sales.sort_values('Sales', ascending=False)['Month Name'].iloc[0]}")
print(f"5. Outlet type with highest closure rate: {outlet_closure.sort_values('Closure Rate', ascending=False)['Outlet Type'].iloc[0]}")
print(f"6. Correlation between store size and sales: {sales_sqft_corr:.4f}")


----- SUMMARY OF KEY INSIGHTS -----
1. Total number of unique stores in the dataset: 830
2. Most profitable Super Division: GIRLS
3. Top 3 states recommended for new stores: E4, B5, C1
4. Best month to open a store: December
5. Outlet type with highest closure rate: Outlet
6. Correlation between store size and sales: 0.4751
